In [1]:
%matplotlib inline
"""
Each of the modules defined in ham1d.models defines a class
for construction of a particular hamiltonian type. All of the
classes share the same base class and thus their functionality
is more or less the same, the difference is in the internal routines
used.
"""

# to load classes utilizing numba's jit
# compilation optimizations, load the following
# modules. The following lines would load modules
# with routines for creation of spin 1/2 (hard-core
# bosonic), spinless fermionic and noninteracting
# hamiltonians.
from ham1d.models import spin1d as sp1
from ham1d.entropy import rdm, ententro

# in case of the spin 1/2 hamiltonians,
# one can also construct the model hamiltonians
# using the kronecker product of sparse operators.
# In general, this could be easily achieved for
# any spin value, simply by providing an appropriate
# set of spin operators. For now, only the spin 1/2
# matrices are provided.

import numpy as np
from scipy import linalg as LA
import matplotlib.pyplot as plt

## Defining the model constants

In [3]:
#first, define the chain length. We choose a rather small system here:
L=10
subsize=int(L*0.5)
# define constants and model parameters:
J=1.
W=1.
Delta = 0.55
gldn = (np.sqrt(5.) - 1.) * 0.5

In [4]:
J_pm1 = [[J * 0.5, i, (i + 1) % L]
         for i in range(L)]  # the %L part ensures PBC
J_pm2 = [[J * 0.5, i, (i + 2) % L] for i in range(L)]

flips = [['+-', J_pm1], ['-+', J_pm1], ['+-', J_pm2], ['-+', J_pm2]]

# interaction:
J_zz1 = [[J * Delta, i, (i + 1) % L] for i in range(L)]
J_zz2 = [[J * Delta, i, (i + 2) % L] for i in range(L)]

inter = [['zz', J_zz1], ['zz', J_zz2]]

# random field

fields = np.array([W * np.cos(2 * np.pi * gldn * i) for i in range(1, L + 1)])
J_z = [[fields[i], i] for i in range(L)]
rnd = ['z', J_z]

# we can now put together the static_list -> just a list of hamiltonian term
# definitions

static_list = [*flips, *inter, rnd]

In [6]:
# first, let us construct a hamiltonian for all the
# spin projection blocks. We do this by selecting Nu=None.
ham = sp1.hamiltonian(L, static_list, [], Nu= int(L/2.))

Please wait, building the Hamiltonian ...
Building the Hamiltonian finished!
Calculating nnz, o_nnz, d_nnz!
Calculating nnz, o_nnz, d_nnz finished!


In [7]:
eigvals, eigvecs = ham.eigsystem()

In [6]:
eigvecs

array([[ 0.00458543-0.j,  0.00053225-0.j,  0.00348589-0.j, ...,
         0.06391164+0.j,  0.03466412+0.j,  0.0262157 +0.j],
       [ 0.00397601+0.j,  0.00090516+0.j,  0.00937632+0.j, ...,
         0.04057223+0.j,  0.02976509+0.j,  0.02405883+0.j],
       [-0.01169399+0.j, -0.0027642 +0.j, -0.00396319+0.j, ...,
         0.09062154+0.j,  0.06436086+0.j,  0.05932476+0.j],
       ...,
       [ 0.01933474+0.j, -0.04469196+0.j,  0.06598708+0.j, ...,
        -0.06436623+0.j, -0.0124998 +0.j,  0.02900159+0.j],
       [-0.00105674+0.j,  0.0055556 +0.j, -0.00919938+0.j, ...,
        -0.11785648+0.j, -0.05207184+0.j,  0.0454983 +0.j],
       [-0.00458839+0.j,  0.00654105+0.j, -0.00356627+0.j, ...,
        -0.06551426+0.j, -0.03363043+0.j,  0.02484149+0.j]])

In [8]:
# check entropy calculations
eigstate = eigvecs[4]

In [8]:
rdm_matrix.todense()

NameError: name 'rdm_matrix' is not defined

In [12]:
#%%timeit
rdm_matrix = rdm.build_rdm(eigstate, subsize, L, int(L/2.))
rdm_eigvals=LA.eigvalsh(rdm_matrix.todense())
rdm_eigvals=rdm_eigvals[rdm_eigvals>1e-014]
entropy=np.dot(rdm_eigvals, np.log(rdm_eigvals))

#print(-entropy)

In [13]:
#%%timeit

# the other way
vec_ = np.zeros_like(ham.state_indices, dtype=np.complex128)
vec_[ham.states] = eigstate

Entangled = ententro.Entangled
ent_object = Entangled(vec_, L, subsize)
ent_object.partitioning('homogenous')
ent_object.svd(full_matrices=True)

eentro = ent_object.eentro()
#print(eentro)

In [14]:
-entropy - eentro

0.0